In [6]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import sounddevice as sd
import numpy as np
import librosa
from tensorflow.keras.models import load_model
import os

# Function to record audio
def record_audio(duration=2, sample_rate=16000):
    print("Recording...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()  # Wait until recording is finished
    print("Recording complete.")
    return audio_data.flatten(), sample_rate

# Normalize audio to range [-1, 1]
def normalize_audio(audio):
    return audio.astype(np.float32) / 32768.0

# Extract MFCC features
def extract_mfcc(audio, sr, duration=2, n_mfcc=40, frames=32):
    audio = normalize_audio(audio)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    if mfcc.shape[1] < frames:
        mfcc = np.pad(mfcc, ((0, 0), (0, frames - mfcc.shape[1])), mode='constant')
    elif mfcc.shape[1] > frames:
        mfcc = mfcc[:, :frames]
    mfcc = np.expand_dims(mfcc, axis=-1)  # Add channel dimension
    mfcc = np.expand_dims(mfcc, axis=0)  # Add batch dimension
    return mfcc

# Recognize command from microphone
def recognize_command(model, commands, duration=2):
    audio, sr = record_audio(duration)
    mfcc = extract_mfcc(audio, sr)
    prediction = model.predict(mfcc)
    predicted_command = commands[np.argmax(prediction)]
    print(f"Recognized Command: {predicted_command}")
    return predicted_command

# Main GUI Application
class VoiceCommandApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Learn App")
        self.root.geometry("600x600")
        
        # Load models
        self.number_model = load_model("numbersv3.h5")  # Trained model for recognizing numbers
        self.motion_model = load_model("motions.h5")  # Trained model for motions (up, down, left, right)
        self.animal_model = load_model("animalsv3.h5")  # Trained model for recognizing animals
        self.number_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Compile model
        self.motion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        self.animal_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.commands = ['up', 'down', 'left', 'right']
        self.number_commands = [str(i) for i in range(10)]  # Numbers 0-9
        self.animal_commands = ['cat', 'dog', 'bird']
        
        # Set background color
        self.bg_image_path = "back.png"  # Replace with your background image file path
        if not os.path.exists(self.bg_image_path):
            raise FileNotFoundError(f"Background image file not found: {self.bg_image_path}")
        
        bg_image = Image.open(self.bg_image_path)
        bg_image = bg_image.resize((600, 600))  # Resize to fit the window size
        self.bg_photo = ImageTk.PhotoImage(bg_image, master=root)
        
        # Start menu with 3 options
        self.create_start_menu()

    def create_start_menu(self):
        # Clear the window to ensure it's empty before showing the menu
        self.clear_window()
        
        # Set background image
        self.set_background()

        # Header
        header_label = tk.Label(self.root, text="Welcome Back!", font=("Helvetica", 30, "bold"), fg="green", bg="#FFFFFF")
        header_label.place(relx=0.5, rely=0.2, anchor='center')  # Center header
        
        # Buttons for options
        learn_numbers_btn = tk.Button(self.root, text="Learn Numbers", font=("Helvetica", 16), bg="#81C784", command=self.learn_numbers)
        learn_numbers_btn.place(relx=0.5, rely=0.4, anchor='center')  # Center button

        learn_motions_btn = tk.Button(self.root, text="Learn Motions", font=("Helvetica", 16), bg="#81C784", command=self.learn_motions)
        learn_motions_btn.place(relx=0.5, rely=0.5, anchor='center')  # Center button

        learn_animals_btn = tk.Button(self.root, text="Learn Animals", font=("Helvetica", 16), bg="#81C784", command=self.learn_animals)
        learn_animals_btn.place(relx=0.5, rely=0.6, anchor='center')  # Center button

    def back_to_start_menu(self):
        # Function to go back to the start menu
        self.create_start_menu()

    def learn_numbers(self):
        # Function for learning numbers
        self.clear_window()  # Clear the window
        self.set_background()

        # Randomly select a number (0-9)
        number = np.random.choice(self.number_commands)
        
        # Display the number image
        number_image_path = f"images/numbers/{number}.png"  # Replace with path to your number images (0.png, 1.png, etc.)
        number_image = Image.open(number_image_path)
        number_image = number_image.resize((150, 150))
        number_photo = ImageTk.PhotoImage(number_image, master=self.root)
        
        number_label = tk.Label(self.root, image=number_photo)
        number_label.image = number_photo  # Keep reference to prevent garbage collection
        number_label.place(relx=0.5, rely=0.3, anchor='center')

        # Prompt to say the number
        prompt_label = tk.Label(self.root, text="Say the number", font=("Helvetica", 18), fg="#81C784", bg="#FFFFFF")
        prompt_label.place(relx=0.5, rely=0.5, anchor='center')

        # Feedback label to show whether it's correct or not
        self.feedback_label = tk.Label(self.root, text="", font=("Helvetica", 16), fg="#FF6347", bg="#FFFFFF")
        self.feedback_label.place(relx=0.5, rely=0.6, anchor='center')

        # Record button
        record_button = tk.Button(self.root, text="Record", font=("Helvetica", 16), bg="#81C784", command=lambda: self.check_number(number, number_label, prompt_label))
        record_button.place(relx=0.5, rely=0.7, anchor='center')
        
        # Back button
        back_button = tk.Button(self.root, text="Back", font=("Helvetica", 16), bg="#FFB6C1", command=self.back_to_start_menu)
        back_button.place(relx=0.5, rely=0.8, anchor='center')

    def check_number(self, number, number_label, prompt_label):
        # Recognize number command
        try:
            command = recognize_command(self.number_model, self.number_commands)
            if command == number:
                self.feedback_label.config(text="Correct!", fg="green")
                self.root.after(2000, self.learn_numbers)  # Wait 2 seconds before changing the number
            else:
                self.feedback_label.config(text="Wrong, try again!", fg="red")
        except Exception as e:
            self.feedback_label.config(text=f"Error: {e}", fg="red")

    def learn_motions(self):
        # Function for learning motions
        self.clear_window()
        self.set_background()

        # Create a smaller canvas to act as the box for Dora (200x200)
        self.canvas = tk.Canvas(self.root, width=200, height=200, bg="#FFFFFF")
        self.canvas.place(relx=0.5, rely=0.4, anchor='center')  # Center canvas

        # Create Dora inside the canvas (initial position)
        self.dora_image_path = "dora.png"  # Replace with your Dora image path
        self.dora_image = Image.open(self.dora_image_path)
        self.dora_image = self.dora_image.resize((80, 80))  # Resize Dora to fit in the smaller box
        self.dora_photo = ImageTk.PhotoImage(self.dora_image)

        # Place Dora inside the box (initial position)
        self.dora_label = self.canvas.create_image(100, 100, image=self.dora_photo)  # Initial position (center of box)
        
        # Display instructions for learning motions
        tk.Label(self.root, text="Say one of : Up, Down, Left, Right", font=("Helvetica", 16), fg="#81C784",bg="#FFFFFF").place(relx=0.5, rely=0.6, anchor='center')
        
        # Record button for motion commands
        record_button = tk.Button(self.root, text="Record", font=("Helvetica", 16), bg="#81C784", command=self.perform_motion)
        record_button.place(relx=0.5, rely=0.7, anchor='center')

        # Back button
        back_button = tk.Button(self.root, text="Back", font=("Helvetica", 16), bg="#FFB6C1", command=self.back_to_start_menu)
        back_button.place(relx=0.5, rely=0.8, anchor='center')

    def perform_motion(self):
        # Recognize motion command
        try:
            command = recognize_command(self.motion_model, self.commands)
            if command == 'up':
                self.move_dora("up")
                #self.feedback_label.config(text="Correct, Dora is moving Up!", fg="green")
            elif command == 'down':
                self.move_dora("down")
               # self.feedback_label.config(text="Correct, Dora is moving Down!", fg="green")
            elif command == 'left':
                self.move_dora("left")
                #self.feedback_label.config(text="Correct, Dora is moving Left!", fg="green")
            elif command == 'right':
                self.move_dora("right")
               # self.feedback_label.config(text="Correct, Dora is moving Right!", fg="green")
            else:
                self.feedback_label.config(text="Wrong, try again!", fg="red")
        except Exception as e:
            self.feedback_label.config(text=f"Error: {e}", fg="red")

    def move_dora(self, direction):
        # Get current coordinates of Dora
        current_x, current_y = self.canvas.coords(self.dora_label)

        # Define the smaller box boundaries (200x200)
        box_left = 0
        box_top = 0
        box_right = 200
        box_bottom = 200

        # Move Dora according to the command, within the box boundaries
        if direction == "up" and current_y > 50:
            self.canvas.move(self.dora_label, 0, -50)  # Move Dora up
        elif direction == "down" and current_y < 150:
            self.canvas.move(self.dora_label, 0, 50)  # Move Dora down
        elif direction == "left" and current_x > 50:
            self.canvas.move(self.dora_label, -50, 0)  # Move Dora left
        elif direction == "right" and current_x < 150:
            self.canvas.move(self.dora_label, 50, 0)  # Move Dora right

    def learn_animals(self):
        # Function for learning animals
        self.clear_window()  # Clear the window
        self.set_background()

        # Randomly select an animal
        animal = np.random.choice(self.animal_commands)
        
        # Display the animal image
        animal_image_path = f"images/animals/{animal}.png"  # Replace with path to your animal images (cat.png, dog.png, etc.)
        animal_image = Image.open(animal_image_path)
        animal_image = animal_image.resize((150, 150))
        animal_photo = ImageTk.PhotoImage(animal_image, master=self.root)
        
        animal_label = tk.Label(self.root, image=animal_photo)
        animal_label.image = animal_photo  # Keep reference to prevent garbage collection
        animal_label.place(relx=0.5, rely=0.3, anchor='center')

        # Prompt to say the animal
        prompt_label = tk.Label(self.root, text="Say the animal", font=("Helvetica", 18), fg="#81C784", bg="#FFFFFF")
        prompt_label.place(relx=0.5, rely=0.5, anchor='center')

        # Feedback label to show whether it's correct or not
        self.feedback_label = tk.Label(self.root, text="", font=("Helvetica", 16), fg="#FF6347", bg="#FFFFFF")
        self.feedback_label.place(relx=0.5, rely=0.6, anchor='center')

        # Record button
        record_button = tk.Button(self.root, text="Record", font=("Helvetica", 16), bg="#81C784", command=lambda: self.check_animal(animal, animal_label))
        record_button.place(relx=0.5, rely=0.7, anchor='center')

        # Back button
        back_button = tk.Button(self.root, text="Back", font=("Helvetica", 16), bg="#FFB6C1", command=self.back_to_start_menu)
        back_button.place(relx=0.5, rely=0.8, anchor='center')

    def check_animal(self, animal, animal_label):
        # Recognize animal command
        try:
            command = recognize_command(self.animal_model, self.animal_commands)
            if command == animal:
                self.feedback_label.config(text="Correct!", fg="green")
                self.root.after(2000, self.learn_animals) 
            else:
                self.feedback_label.config(text="Wrong, try again!", fg="red")
        except Exception as e:
            self.feedback_label.config(text=f"Error: {e}", fg="red")

    def set_background(self):
        bg_label = tk.Label(self.root, image=self.bg_photo)
        bg_label.place(relwidth=1, relheight=1)

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# Create the main window
root = tk.Tk()
app = VoiceCommandApp(root)
root.mainloop()

Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
Recognized Command: 5
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Recognized Command: 9
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Recognized Command: 1
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Recognized Command: 2
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Recognized Command: 2
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Recognized Command: 2
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Recognized Command: 4
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Recognized Command: 5
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Recognized Command: 0
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Recognized Command: 6
Recording...
Recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/ste